<a href="https://colab.research.google.com/github/dhruvbaliyan/Object-Detection/blob/main/RealTIme_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "numpy==1.26.4" "tensorflow==2.19.0" "tf-models-official==2.19.1" "protobuf<=3.20.3"


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 102910, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 102910 (delta 160), reused 98 (delta 90), pack-reused 102693 (from 3)
Receiving objects: 100% (102910/102910), 642.87 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (74329/74329), done.


In [3]:
%cd /content/models/research

/content/models/research


In [4]:
!ls

adversarial_text    deeplab		   marco	     seq_flow_lite
attention_ocr	    deep_speech		   nst_blogpost      slim
audioset	    delf		   object_detection  vid2depth
autoaugment	    efficient-hrl	   pcl_rl
cognitive_planning  lfads		   README.md
cvt_text	    lstm_object_detection  rebar


In [5]:
!apt-get install -y protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.4).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [7]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-model-optimization to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow-model-optimization to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking 

In [8]:
import os
import pathlib
import time
import tensorflow as tf
import imageio

import numpy as np
import matplotlib.pyplot as plt

from object_detection.utils import \
  label_map_util, visualization_utils as viz_utils

from PIL import Image
%matplotlib inline

In [21]:
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

In [22]:
PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname= 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin=url,
    untar=True
)
PATH_TO_MODEL_DIR

'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

In [26]:
!ls -F {PATH_TO_MODEL_DIR}

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/


In [24]:
url1 = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'


In [25]:
PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url1,
    untar=False)

5056/5056 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [28]:
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


In [29]:
MODEL_DIR = os.path.join(PATH_TO_MODEL_DIR, "ssd_resnet101_v1_fpn_640x640_coco17_tpu-8")
PATH_TO_SAVED_MODEL = os.path.join(MODEL_DIR, "saved_model")

In [30]:
!ls {PATH_TO_SAVED_MODEL}

assets	saved_model.pb	variables


In [31]:
print('Loading model...', end='')
start_time = time.time()

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()

elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 76.54938507080078 seconds


In [32]:
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS,
    use_display_name=True)

In [33]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

#### Main Realtime detection

In [34]:
import cv2

In [35]:
def detectObject(frame_bgr):
  image_np = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
  # OpenCV captures images in BGR format, but TensorFlow’s model expects RGB color ordering.
  # ✔ Convert BGR → RGB
  input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

  detections = detect_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))
  detections = {k: v[0, :num_detections].numpy() for k, v in detections.items()}
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_np_with_detections = image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=50,
    min_score_thresh=0.5,
    agnostic_mode=False
  )
  return cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)


In [39]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    out_frame = detectObject(frame)
    cv2.imshow("Real-time detection", out_frame)

    # quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
